# Integer Optimization with Pyomo

In [1]:
#import pyomo
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Finding Weather Data for Munich

In [2]:
import requests, json


In [3]:
api_key = "c89d098c41c75d31f57ca2a4b12d0eeb"
api_key2= "f60822f9e7b31667c7fbdd53d73977b8"
base_url = "http://api.openweathermap.org/data/2.5/weather?"


In [4]:
# Give city name
city_name = input("Munich")

In [5]:
#initialize variable temperature_celsius
temperature_celsius = 0


In [6]:
def get_weather(city):
    global temperature_celsius
    global humidity
    api_key = 'f60822f9e7b31667c7fbdd53d73977b8'
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

    try:
        response = requests.get(url)
        data = response.json()

        if response.status_code == 200:
            weather = data['weather'][0]['description']
            temperature = data['main']['temp']
            temperature_celsius = temperature - 273.15              
            temperature_celsius = round(temperature_celsius, 0)
            #convert temperature celsius to integer
            temperature_celsius = int(temperature_celsius)
            humidity = data['main']['humidity']
            humidity = float(humidity)


            print(f"Weather in {city}: {weather}")
            print(f"Temperature: {temperature_celsius} C")
            print(f"Humidity: {humidity}%")

        else:
            print(f"Error: {data['message']}")
            
    except requests.exceptions.RequestException as e:
        print(f"Error: {str(e)}")



In [7]:

# Example usage
get_weather('Munich')

Weather in Munich: clear sky
Temperature: 17 C
Humidity: 63.0%


In [8]:
temperature_celsius

17

In [9]:
humidity

63.0

In [36]:

def get_demands_by_temperature(temperature_celsius):
    global wine_demand
    global ice_cream_demand
    global coffee_demand
    #make
    

    if temperature_celsius <= 10:
        wine_demand = int(80)
        ice_cream_demand = int(5)
        coffee_demand = int(60)
    elif temperature_celsius > 10 and temperature_celsius <= 20:
        wine_demand = int(30)
        ice_cream_demand = int(20)
        coffee_demand = int(80)
    elif temperature_celsius > 20:
        wine_demand = int(5)
        ice_cream_demand = int(80)
        coffee_demand = int(40)

    return wine_demand, ice_cream_demand, coffee_demand

In [37]:
get_demands_by_temperature(temperature_celsius)

(30, 20, 80)

In [38]:
wine_demand

30

# Creating seasons and demand 

In [39]:
# create three variables for hot_wine, coffee and ice_cream with bounds between 0 and 40 which must be random
hot_wine = pyo.Var(within=pyo.NonNegativeReals, bounds=(0,100), initialize= wine_demand)
coffee = pyo.Var(within=pyo.NonNegativeReals, bounds=(0,100), initialize= coffee_demand)
ice_cream = pyo.Var(within=pyo.NonNegativeReals, bounds=(0,100), initialize= ice_cream_demand)

## Creating a concrete model

The concrete model is to represent the optimization problem

In [40]:
# Create a concrete model
model = pyo.ConcreteModel()

In [43]:
#show model 
model.pprint()

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 2*AbstractScalarVar + 3*AbstractScalarVar + AbstractScalarVar

1 Declarations: obj


## Define the Objective Function

These are the profits

In [44]:
#define the objective function
model.obj = pyo.Objective(expr = 2*hot_wine + 3*coffee + ice_cream, sense = maximize)

'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [45]:
#show the objective function
model.pprint()

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 2*AbstractScalarVar + 3*AbstractScalarVar + AbstractScalarVar

1 Declarations: obj


## Define the Constrains

These are the weights

In [46]:
#capacity constraint for hot_wine and ice_cream and coffee
model.hot_wine_con = pyo.Constraint(expr = hot_wine <= 120)
model.coffee_con = pyo.Constraint(expr = coffee <= 200)
model.ice_cream_con = pyo.Constraint(expr = ice_cream <= 70)

In [47]:
instance = model.create_instance()  # Create a concrete instance of the model

In [48]:
#create different solvers and solve the model
opt = SolverFactory('cbc')
results = opt.solve(instance) # solves and updates instance
instance.display() # display solution
  
# print the values of the variables
print("The value of hot_wine is: " + str(pyo.value(instance.hot_wine)))
print("The value of coffee is: " + str(pyo.value(instance.coffee)))
print("The value of ice_cream is: " + str(pyo.value(instance.ice_cream)))
  
# print the values of the objective function
print("The value of the objective function is: " + str(pyo.value(instance.obj)))

 

RuntimeError: Cannot access property 'fixed' on AbstractScalarVar 'AbstractScalarVar' before it has been constructed (initialized): 'fixed' is an attribute on an Abstract component and cannot be accessed until the component has been fully constructed (converted to a Concrete component) using AbstractModel.create_instance() or AbstractScalarVar.construct().

In [30]:
#use  using AbstractModel.create_instance() or AbstractScalarVar.construct() to create a concrete instance of the model
opt = SolverFactory('glpk') #select solver
results = opt.solve(instance) #solve the problem
instance.display() #display the results

RuntimeError: Cannot access property 'fixed' on AbstractScalarVar 'AbstractScalarVar' before it has been constructed (initialized): 'fixed' is an attribute on an Abstract component and cannot be accessed until the component has been fully constructed (converted to a Concrete component) using AbstractModel.create_instance() or AbstractScalarVar.construct().

## Step 5: Solve the problem

In [17]:
# Solve the optimization problem with bounds and constraints
opt = SolverFactory('glpk')
results = opt.solve(model) # solves and updates instance
model.pprint()

RuntimeError: Cannot access property 'fixed' on AbstractScalarVar 'AbstractScalarVar' before it has been constructed (initialized): 'fixed' is an attribute on an Abstract component and cannot be accessed until the component has been fully constructed (converted to a Concrete component) using AbstractModel.create_instance() or AbstractScalarVar.construct().

## Step 6: Solution

In [ ]:
# Print the results of the optimization problem
print('x1 = ', pyo.value(model.x1))
print('x2 = ', pyo.value(model.x2))
print('x3 = ', pyo.value(model.x3))
print('Objective = ', pyo.value(model.obj))
 
   

In [ ]:
results.write()
